In [242]:
import pandas as pd
import os
os.chdir("../input")

In [243]:
tf = pd.read_csv("MTeams.csv")
cities_df = pd.read_csv("Cities.csv")


In [244]:
#rf.WLoc = rf["WLoc"].replace("N",0).replace("H",1).replace("A",0)
#rf["score"] = .5 + (rf["WScore"]-rf["LScore"])*2/(rf["WScore"]) - rf.WLoc/20
#rf.score = rf.score.apply(lambda x: min(x,1)).apply(lambda x: max(x,.5))
#X = X.select_dtypes("int")

In [245]:
def convert_int(x):
    try:
        return int(x)
    except ValueError:
        return int(x[:-1])

In [272]:
import random
train = pd.read_csv("MNCAATourneyCompactResults.csv")
seed_df = pd.read_csv("MNCAATourneySeeds.csv")
train = train.rename(columns={"WTeamID":"ATeamID","WScore":"AScore","LTeamID":"BTeamID","LScore":"BScore"})
train["y"] = 1

In [273]:

split = [True if random.randint(0,1)==1 else False for x in range(len(train))]
train["split"] = split
switch_train = train[~train["split"]].drop(columns=["split"])
train = train[train["split"]].drop(columns=["split"])
switch_train = switch_train.rename(columns={"BLoc":"ALoc","ATeamID":"BTeamID","AScore":"BScore","BTeamID":"ATeamID","BScore":"AScore"})

In [274]:
#switch_train = train.copy() #- OVERFIT
#switch_train = switch_train.rename(columns={"BLoc":"ALoc","ATeamID":"BTeamID","AScore":"BScore","BTeamID":"ATeamID","BScore":"AScore"})

In [275]:
switch_train.y = 0
train = pd.concat([train,switch_train])
train = train.merge(seed_df, left_on=["ATeamID","Season"], right_on=["TeamID","Season"])
train = train.merge(seed_df, left_on=["BTeamID","Season"], right_on=["TeamID","Season"])
train = train.rename(columns={"Seed_x":"ASeed","Seed_y":"BSeed"}).drop(columns=["TeamID_x","TeamID_y"])
train.loc[:,["AZone","ASeed"]] = list(train.ASeed.apply(lambda x: [x[0],convert_int(x[1:])]))
train.loc[:,["BZone","BSeed"]] = list(train.BSeed.apply(lambda x: [x[0],convert_int(x[1:])]))

In [276]:
y_train = train["y"]
train = train[["Season","ATeamID","BTeamID","ASeed","BSeed"]] #,"AZone","BZone"


In [277]:
#X[["ATeamID","BTeamID"]] = X[["ATeamID","BTeamID"]].astype("object")
#X[["ATeamID","BTeamID"]] = X[["ATeamID","BTeamID"]].astype("object")
test = pd.read_csv("../input/MSampleSubmissionStage1.csv")
test_ID = test.ID
test = pd.DataFrame(list(test.ID.apply(lambda x: x.split("_"))), test_ID, columns=["Season","ATeamID","BTeamID"]).astype("int").reset_index()
test = test.merge(seed_df, left_on=["ATeamID","Season"], right_on=["TeamID","Season"])
test = test.merge(seed_df, left_on=["BTeamID","Season"], right_on=["TeamID","Season"])
test = test.rename(columns={"Seed_x":"ASeed","Seed_y":"BSeed"}).drop(columns=["TeamID_x","TeamID_y"])
test.loc[:,["AZone","ASeed"]] = list(test.ASeed.apply(lambda x: [x[0],convert_int(x[1:])]))
test.loc[:,["BZone","BSeed"]] = list(test.BSeed.apply(lambda x: [x[0],convert_int(x[1:])]))
test = test.set_index("ID").loc[test_ID]

test = test[["Season","ATeamID","BTeamID","ASeed","BSeed"]]




In [278]:
X = pd.concat([train,test])
X = pd.get_dummies(X)
train = X.iloc[:len(y_train)]
test = X.iloc[len(y_train):]

In [279]:
from sklearn.utils import shuffle
train, y_train = shuffle(train, y_train)


In [280]:
#X[X.isnull().any(axis=1)]

In [281]:
import numpy as np
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold, cross_val_score, train_test_split 
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler # Preprocessing
seed = 5
n_folds =5
scoring='neg_mean_squared_error'
model = LGBMRegressor()
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
np.sqrt(-cross_val_score(model, train, y_train, cv= kfold,
                                 scoring=scoring, n_jobs=1))

array([0.43220734, 0.45246755, 0.44262991, 0.49937151, 0.48361559,
       0.47039382, 0.451105  , 0.46305372, 0.44672685, 0.45010607])

In [282]:
model.fit(train, y_train)

LGBMRegressor()

In [178]:
import pickle
pkl_filename = "../output/lgbm.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

In [283]:
finalPred = model.predict(test)
finalPred = [min(max(x,0),1) for x in finalPred]

In [284]:
sub = pd.DataFrame()
sub['ID'] = test_ID
sub['Pred'] = finalPred
#sub.to_csv('../output/lgb_submission.csv',index=False)

In [285]:
from sklearn.metrics import mean_squared_error as MSE
predictions = sub["Pred"]
y_test = np.log1p(pd.read_csv("../output/lgb_submission.csv")["Pred"])
rmse = np.sqrt(MSE(y_test, predictions)) 
print("RMSE : % f" %(rmse)) 

RMSE :  0.214457


# Add in Real Data to model Data

In [155]:
train = pd.read_csv("MNCAATourneyCompactResults.csv")
train.WLoc = train["WLoc"].replace("N",0).replace("H",1).replace("A",0)
train["score"] = .5 + (train["WScore"]-train["LScore"])*2/(train["WScore"]) - train.WLoc/20
train.score = train.score.apply(lambda x: min(x,1)).apply(lambda x: max(x,.5))
train = train.rename(columns={"WTeamID":"ATeamID","LTeamID":"BTeamID"})[["Season","ATeamID","BTeamID","score"]]

switch_train = train.rename(columns={"ATeamID":"BTeamID","BTeamID":"ATeamID"})
switch_train["score"] = 1 - switch_train["score"]
train = pd.concat([train,switch_train])

real=test.reset_index().merge(train, on=["ATeamID","BTeamID","Season"])
real = real.groupby(["ATeamID","BTeamID","Season"]).agg({"score":'mean','index':'first'}).reset_index()

In [158]:

m_test = test.drop(real["index"])
finalPred = model.predict(m_test)
finalPred = [min(max(x,0),1) for x in finalPred]

model_ID = m_test.index
real_ID = real["index"]

sub = pd.DataFrame()
sub['ID'] = list(model_ID) + list(real_ID)
sub['Pred'] = list(finalPred) + list(real["score"])
sub.set_index("ID").loc[test_ID]

,Pred
ID,
2015_1107_1112,0.000000
2015_1107_1116,0.000000
2015_1107_1124,0.000000
2015_1107_1125,0.384215
2015_1107_1129,0.183506
...,...
2019_1449_1459,0.289626
2019_1449_1463,0.733976
2019_1458_1459,0.286167


In [159]:
sub.to_csv("../output/LGBM-real.csv",index=False)

In [153]:
real

,ATeamID,BTeamID,Season,y,index
0,1101,1246,2019,0,2019_1101_1246
1,1104,1437,2018,0,2018_1104_1437
2,1104,1439,2018,1,2018_1104_1439
3,1107,1328,2015,0,2015_1107_1328
4,1112,1138,2018,0,2018_1112_1138
...,...,...,...,...,...
330,1437,1452,2018,1,2018_1437_1452
331,1437,1458,2017,0,2017_1437_1458
332,1439,1458,2017,0,2017_1439_1458
333,1451,1462,2016,0,2016_1451_1462


In [1]:
sub.Season.unique()

NameError: name 'sub' is not defined